In [4]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import warnings
warnings.filterwarnings("ignore")

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

In [5]:
# HELPER METHODS
import samplerate
import scipy

def get_time_segments(subject, sr_to_convert=0):
    data = dr.get_participant_data(subject)
    labels = data["label"]
    if sr_to_convert > 0:
        step = int(700/sr_to_convert)
        resampled = []
        for i in range(0, labels.size, step):
            resampled.append(labels[i])
        labels = np.asarray(resampled)
        # labels = samplerate.resample(labels, sr_to_convert/700.0)
        # labels = scipy.signal.decimate(labels, int(700.0/sr_to_convert))
    curr_value = labels[0]
    start_indices = [0]
    end_indices = []
    i = 1
    while i < len(labels):
        if labels[i] != curr_value:
            end_indices.append(i-1)
            start_indices.append(i)
            curr_value = labels[i]
        i += 1

    end_indices.append(len(labels)-1)

    time_segments = {}
    for i in range(len(start_indices)):
        label = labels[start_indices[i]]
        if label not in time_segments.keys():
            time_segments[label] = [(start_indices[i], end_indices[i])]
        else:
            time_segments[label].append((start_indices[i], end_indices[i]))
    return time_segments


# ts = get_time_segments(2)
# print(ts)

In [11]:
# SAVE SIGNALS INTO INDIVIDUAL CSV FILES
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


subject_indices = list(range(2, 12)) + list(range(13, 18))
subject_indices = [str(i) for i in subject_indices]

locations = [dr.WESADKeys.CHEST]
# locations = [dr.WESADKeys.WRIST]
chest_signals = [
    dr.Signals.ACC, dr.Signals.ECG, dr.Signals.EDA, dr.Signals.EMG, "Resp", "Temp"
]
wrist_signals = [
    dr.Signals.ACC, dr.Signals.BVP, dr.Signals.EDA, dr.Signals.TEMP
]
phase_label_dict = {
    1: dr.Phases.BASE,
    2: dr.Phases.TSST,
    3: dr.Phases.FUN,
    4: [dr.Phases.MEDI_1, dr.Phases.MEDI_2]
}

phases = dr.Phases.PHASE_ORDER
# location = "wrist"
location = "chest"

for subject in subject_indices:
    data = dr.get_participant_data(subject)
    
    for signal in chest_signals:
    # for signal in wrist_signals:
        if location == "wrist":
            sr = dr.FS_DICT["wrist"][signal]
        time_segments = get_time_segments(subject, sr)
        signal_data = data["signal"][location][signal]
        for label in list(phase_label_dict.keys()):
            if signal == "Temp" or signal == "Resp":
                signal = signal.upper()
            phase = phase_label_dict[label]
            if label == 4:  # 4 corresponds to MEDI 1 and MEDI 2; includes 2 time segment tuples
                if signal == dr.Signals.ACC:
                    columns_1 = [f"{location}_{signal}_X_{phase[0]}", f"{location}_{signal}_Y_{phase[0]}", f"{location}_{signal}_Z_{phase[0]}"]
                else: 
                    columns_1 = [f"{location}_{signal}_{phase[0]}"]
                file_name_1 = f"S{subject}_{location}_{signal}_{phase[0]}.csv"
                file_name_1 = os.path.join(dr.Paths.WESAD, f"S{subject}", file_name_1)
                start = time_segments[label][0][0]
                end = time_segments[label][0][1]
                phase_data = signal_data[start:end]
                phase_data = pd.DataFrame(data=phase_data, columns=columns_1)
                phase_data.to_csv(file_name_1)

                if signal == dr.Signals.ACC:
                    columns_2 = [f"{location}_{signal}_X_{phase[1]}", f"{location}_{signal}_Y_{phase[1]}", f"{location}_{signal}_Z_{phase[1]}"]
                else: 
                    columns_2 = [f"{location}_{signal}_{phase[1]}"]
                file_name_2 = f"S{subject}_{location}_{signal}_{phase[1]}.csv"
                file_name_2 = os.path.join(dr.Paths.WESAD, f"S{subject}", file_name_2)
                start = time_segments[label][1][0]
                end = time_segments[label][1][1]
                phase_data = signal_data[start:end]
                phase_data = pd.DataFrame(data=phase_data, columns=columns_2)
                phase_data.to_csv(file_name_2)
            else:
                if signal == dr.Signals.ACC:
                    columns = [f"{location}_{signal}_X_{phase}", f"{location}_{signal}_Y_{phase}", f"{location}_{signal}_Z_{phase}"]
                else: 
                    columns = [f"{location}_{signal}_{phase}"]
                file_name = f"S{subject}_{location}_{signal}_{phase}.csv"
                file_name = os.path.join(dr.Paths.WESAD, f"S{subject}", file_name)
                start = time_segments[label][0][0]
                end = time_segments[label][0][1]
                phase_data = signal_data[start:end]
                phase_data = pd.DataFrame(data=phase_data, columns=columns)
                phase_data.to_csv(file_name)


KeyError: 'ECG'

In [ ]:
# ECG METRICS
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import heartpy as hp

phases = dr.Phases.PHASE_ORDER
metrics = ["bpm", "rmssd", "ibi"]
subject_indices = [3]
subject_indices = [str(i) for i in subject_indices]
fs = 700.0

for phase in phases:
    data_files = {
        s: dr.Paths.WESAD + f"/S{s}/S{s}_chest_ECG_{phase}.csv" for s in subject_indices
    }
    df_dict = {s: pd.read_csv(data_files[s]) for s in list(data_files.keys())}
    metrics_dict = {
        "bpm": [],
        "rmssd": [],
        "ibi": []
    }
    for s in list(df_dict.keys()):
        df = df_dict[s]
        ecg_signal = df.iloc[:, -1]
        ecg_signal = preprocessing.clean_ecg(ecg_signal).to_numpy().flatten()
        ecg_signal = hp.enhance_ecg_peaks(ecg_signal, fs)
        working_data, measures = hp.process_segmentwise(ecg_signal, fs, segment_width=55, segment_overlap=5/55)
        print(np.isnan(measures["bpm"][0]))
        print(measures["bpm"][0])

In [4]:
# ECG METRICS
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

import heartpy as hp

phases = dr.Phases.PHASE_ORDER
# metrics = ["bpm", "rmssd", "ibi"]
metrics = ["sdnn", "breathingrate"]
subject_indices = dr.SUBJECTS
fs = 700.0

for phase in phases:
    data_files = {
        s: dr.Paths.WESAD + f"/S{s}/S{s}_chest_ECG_{phase}.csv" for s in subject_indices
    }
    df_dict = {s: pd.read_csv(data_files[s]) for s in list(data_files.keys())}
    metrics_dict = {
        # "bpm": [],
        # "rmssd": [],
        # "ibi": [],
        "sdnn": [],
        "breathingrate": []
    }
    for s in list(df_dict.keys()):
        df = df_dict[s]
        ecg_signal = df.iloc[:, -1]
        ecg_signal = preprocessing.clean_ecg(ecg_signal).to_numpy().flatten()
        working_data, measures = hp.process_segmentwise(ecg_signal, fs, segment_width=55, segment_overlap=5/55)
        threshold = 180
        # if any(bpm > threshold for bpm in measures["bpm"]):
        #     print(f"BPM for subject {s} contains values greater than {threshold}") 
        #     valid_mean = np.mean([bpm for bpm in measures["bpm"] if bpm < threshold])
        #     for i in range(len(measures["bpm"])):
        #         if measures["bpm"][i] > threshold:
        #             measures["bpm"][i] = valid_mean
        for metric in list(metrics_dict.keys()):
            if np.isnan(np.sum(measures[metric])):
                mean = np.nanmean(measures[metric])
                measures[metric] = np.nan_to_num(measures[metric], nan=mean).tolist()
            measures[metric].insert(0, int(s))
            metrics_dict[metric].append(pd.DataFrame(measures[metric]))
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df
    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_{phase}.csv")
        metrics_dict[metric].to_csv(file_name)
        
# for phase in phases:
#     data_2 = dr.get_data_for_phase(subject, phase, location, dr.Signals.ECG)
#     fs = dr.FS_DICT[location][modality]
#     wd, m = hp.process(data_2, sample_rate=fs)
#     hp.plotter(wd, m, title=f"Subject {subject}, {phase}, {modality} {location}")


In [ ]:
# HF and LF metrics
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

phases = dr.Phases.PHASE_ORDER
subject_indices = dr.SUBJECTS
fs = 700.0

for phase in phases:
    data_files = {
        s: dr.Paths.WESAD + f"/S{s}/S{s}_chest_ECG_{phase}.csv" for s in subject_indices
    }
    df_dict = {s: pd.read_csv(data_files[s]) for s in list(data_files.keys())}
    metrics_dict = {
        "lf_rr": [],
        "hf_rr": [],
    }
    for s in list(df_dict.keys()):
        df = df_dict[s]
        ecg_signal = df.iloc[:, -1]
        ecg_signal = preprocessing.clean_ecg(ecg_signal)
        lf_rr = preprocessing.get_lf_rr(ecg_signal, fs)
        lf_rr = np.insert(lf_rr, 0, s)
        lf_rr = pd.DataFrame(lf_rr).dropna(axis=1)

        hf_rr = preprocessing.get_hf_rr(ecg_signal, fs)
        hf_rr = np.insert(hf_rr, 0, s)
        hf_rr = pd.DataFrame(hf_rr).dropna(axis=1)

        metrics_dict["lf_rr"].append(lf_rr)
        metrics_dict["hf_rr"].append(hf_rr)
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df
    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_{phase}.csv")
        metrics_dict[metric].to_csv(file_name)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zhoux\\Desktop\\Projects\\anxiety\\data\\WESAD/S2/S2_chest_ECG_Base.csv'

In [4]:
# FEATURE EXTRACTION - EDA
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


phases = dr.Phases.PHASE_ORDER
subject_indices = dr.SUBJECTS
fs = 700.0

for phase in phases:
    data_files = {
        s: dr.Paths.WESAD + f"/S{s}/S{s}_chest_EDA_{phase}.csv" for s in subject_indices
    }
    df_dict = {s: pd.read_csv(data_files[s]) for s in list(data_files.keys())}
    metrics_dict = {
        "mean_SCL": [],
        "SCR_rate": [],
    }
    for s in list(df_dict.keys()):
        df = df_dict[s]
        eda_signal = df.iloc[:, -1]
        mean_scl = preprocessing.get_mean_SCL(eda_signal, fs)
        mean_scl = np.insert(mean_scl, 0, s)
        mean_scl = pd.DataFrame(mean_scl).dropna(axis=1)

        scr_rate = preprocessing.get_SCR_rate(eda_signal, fs)
        scr_rate = np.insert(scr_rate, 0, s)
        scr_rate = pd.DataFrame(scr_rate).dropna(axis=1)

        metrics_dict["mean_SCL"].append(mean_scl)
        metrics_dict["SCR_rate"].append(scr_rate)
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df
    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_{phase}.csv")
        metrics_dict[metric].to_csv(file_name)

In [6]:
# FEATURE EXTRACTION - WRIST ACC
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


phases = dr.Phases.PHASE_ORDER
subject_indices = dr.SUBJECTS

for phase in phases:
    data_files = {
        s: dr.Paths.WESAD + f"/S{s}/S{s}_wrist_ACC_{phase}.csv" for s in subject_indices
    }
    df_dict = {s: pd.read_csv(data_files[s]) for s in list(data_files.keys())}
    metrics_dict = {
        "peak_wrist_acc": [],
        "mean_wrist_activity": [],
    }
    for s in list(df_dict.keys()):
        df = df_dict[s]
        data = df.iloc[:, 1:]
        peak_acc = preprocessing.get_peak_acc_value(data, acc_type="wrist")
        peak_acc = np.insert(peak_acc, 0, s)
        peak_acc = pd.DataFrame(peak_acc).dropna(axis=1)

        mean_act = preprocessing.get_mean_ankle_activity(data)
        mean_act = np.insert(mean_act, 0, s)
        mean_act = pd.DataFrame(mean_act).dropna(axis=1)

        metrics_dict["peak_wrist_acc"].append(peak_acc)
        metrics_dict["mean_wrist_activity"].append(mean_act)
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df
    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_{phase}.csv")
        metrics_dict[metric].to_csv(file_name)

In [7]:
# FEATURE EXTRACTION - TORSO ACC
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


phases = dr.Phases.PHASE_ORDER
subject_indices = dr.SUBJECTS
fs = 700.0

for phase in phases:
    data_files = {
        s: dr.Paths.WESAD + f"/S{s}/S{s}_chest_ACC_{phase}.csv" for s in subject_indices
    }
    df_dict = {s: pd.read_csv(data_files[s]) for s in list(data_files.keys())}
    metrics_dict = {
        "mean_post": [],
        "mean_post_act": [],
    }
    for s in list(df_dict.keys()):
        df = df_dict[s]
        data = df.iloc[:, 1:]
        mean_post = preprocessing.get_mean_posture(data)
        mean_post = np.insert(mean_post, 0, s)
        mean_post = pd.DataFrame(mean_post).dropna(axis=1)

        mean_act = preprocessing.get_mean_activity_torso(data)
        mean_act = np.insert(mean_act, 0, s)
        mean_act = pd.DataFrame(mean_act).dropna(axis=1)

        metrics_dict["mean_post"].append(mean_post)
        metrics_dict["mean_post_act"].append(mean_act)
    for metric in list(metrics_dict.keys()):
        df = pd.concat(metrics_dict[metric], axis=1)
        df = df.transpose()
        metrics_dict[metric] = df
    for metric in list(metrics_dict.keys()):
        file_name = os.path.join(dr.Paths.METRICS, f"{metric}_{phase}.csv")
        metrics_dict[metric].to_csv(file_name)